I tried to create dataset using the equations

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Function defining the TMDD model equations
def tmdd_model(y, t, CL, kint, kout, kdeg, kon, koff):
    A, T = y

    dAdt = -CL + kint * T - kout * A
    dTdt = -kdeg * T + kon * A - koff * (kon * A * T) / (koff + kdeg)


    return [dAdt, dTdt]

# Function for simulating PK/PD
def simulate_pkpd(CL, kint, kout, kdeg, kon, koff, initial_conditions, time_points):
    params = (CL, kint, kout, kdeg, kon, koff)

    # Solve the differential equations
    solution = odeint(tmdd_model, initial_conditions, time_points, args=params)

    return solution[:, 0], solution[:, 1]

# Function for curve fitting
def fit_pkpd(time_points, antibody_exp, target_exp):
    def fit_func(time, CL, kint, kout, kdeg, kon, koff):
        initial_conditions = [antibody_exp[0], target_exp[0]]
        antibody_sim, target_sim = simulate_pkpd(CL, kint, kout, kdeg, kon, koff, initial_conditions, time)
        return np.concatenate((antibody_sim, target_sim))

    popt, _ = curve_fit(fit_func, time_points, np.concatenate((antibody_exp, target_exp)))
    CL_fit, kint_fit, kout_fit, kdeg_fit, kon_fit, koff_fit = popt

    return CL_fit, kint_fit, kout_fit, kdeg_fit, kon_fit, koff_fit

# Parameters for simulation
CL_true = 0.5   # True clearance rate of the antibody
kint_true = 0.2   # True internalization rate constant
kout_true = 0.1   # True elimination rate constant
kdeg_true = 0.05  # True degradation rate constant
kon_true = 0.3    # True association rate constant between antibody and target
koff_true = 0.2   # True dissociation rate constant between antibody-target complex

initial_conditions = [1.0, 0.5]  # Initial antibody and target concentrations
time_points = np.linspace(0, 10, 100)  # Time points for simulation

# Simulate PK/PD with true parameters
antibody_conc_true, target_conc_true = simulate_pkpd(CL_true, kint_true, kout_true, kdeg_true, kon_true, koff_true, initial_conditions, time_points)

# Generate noisy experimental data
noise_std = 0.05  # Standard deviation of noise
antibody_exp = antibody_conc_true + np.random.normal(0, noise_std, size=time_points.shape)
target_exp = target_conc_true + np.random.normal(0, noise_std, size=time_points.shape)

# Fit the model to experimental data
CL_fit, kint_fit, kout_fit, kdeg_fit, kon_fit, koff_fit = fit_pkpd(time_points, antibody_exp, target_exp)

# Simulate PK/PD with fitted parameters
antibody_conc_fit, target_conc_fit = simulate_pkpd(CL_fit, kint_fit, kout_fit, kdeg_fit, kon_fit, koff_fit, initial_conditions, time_points)


In [ ]:
antibody_conc_fit

array([ 1.00000000e+00,  8.56883821e-01,  7.28663830e-01,  6.12899080e-01,
        5.07598887e-01,  4.11126016e-01,  3.22123647e-01,  2.39459395e-01,
        1.62181916e-01,  8.94868174e-02,  2.06896021e-02, -4.47959843e-02,
       -1.07475463e-01, -1.67787876e-01, -2.26117501e-01, -2.82803534e-01,
       -3.38148164e-01, -3.92423385e-01, -4.45876764e-01, -4.98736408e-01,
       -5.51215287e-01, -6.03515047e-01, -6.55829458e-01, -7.08347570e-01,
       -7.61256700e-01, -8.14745313e-01, -8.69005899e-01, -9.24237917e-01,
       -9.80650902e-01, -1.03846782e+00, -1.09792882e+00, -1.15929543e+00,
       -1.22285546e+00, -1.28892875e+00, -1.35787396e+00, -1.43009684e+00,
       -1.50606030e+00, -1.58629680e+00, -1.67142389e+00, -1.76216376e+00,
       -1.85936818e+00, -1.96405077e+00, -2.07742902e+00, -2.20097998e+00,
       -2.33651508e+00, -2.48628203e+00, -2.65310618e+00, -2.84059056e+00,
       -3.05340463e+00, -3.29771182e+00, -3.58182019e+00, -3.91720474e+00,
       -4.32017544e+00, -

# Trying with real dataset :

Source of the dataset :

https://academic.oup.com/nar/article/49/D1/D1358/5957165?searchresult=1#supplementary-data

https://pk-db.com/data?tab=outputs

In [ ]:
from google.colab import drive
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Warfarin-drug/Physics infoemed DeepONet/Real data with PIDeepONets/outputs.csv')

<ipython-input-2-cf4de96286bb>:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Warfarin-drug/Physics infoemed DeepONet/Real data with PIDeepONets/outputs.csv')


In [ ]:
df

,Unnamed: 0,study_name,measurement_type,tissue,calculation_type,sd,se,min,group_pk,output_pk,...,max,substance,label,individual_pk,unit,cv,median,mean,time,choice
0,0,Lowenthal1976,tmax,plasma,NaN,NaN,NaN,NaN,NaN,344,...,NaN,apap,NaN,10.0,hour,NaN,NaN,NaN,NaN,NaN
1,1,Lowenthal1976,kel,plasma,NaN,NaN,NaN,NaN,NaN,342,...,NaN,apap,NaN,10.0,1 / minute,NaN,NaN,NaN,NaN,NaN
2,2,Lowenthal1976,cmax,plasma,NaN,NaN,NaN,NaN,NaN,341,...,NaN,apap,NaN,10.0,gram / liter,NaN,NaN,NaN,NaN,NaN
3,3,Lowenthal1976,thalf,plasma,NaN,NaN,NaN,NaN,NaN,343,...,NaN,apap,NaN,10.0,hour,NaN,NaN,NaN,NaN,NaN
4,4,Lowenthal1976,auc-inf,plasma,NaN,NaN,NaN,NaN,NaN,340,...,NaN,apap,NaN,10.0,gram * hour / liter,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211268,211268,Bae2014,concentration,plasma,sample-mean,0.000168,0.000097,NaN,2279.0,273813,...,NaN,metoprolol,Fig3_met10,NaN,gram / liter,1.120702,NaN,0.000150,10.0,NaN
211269,211269,Bae2014,concentration,plasma,sample-mean,0.000031,0.000015,NaN,2278.0,273825,...,NaN,alpha-hydroxymetoprolol,Fig3_hmt1,NaN,gram / liter,1.318690,NaN,0.000023,8.0,NaN
211270,211270,Bae2014,concentration,plasma,sample-mean,0.000089,0.000044,NaN,2278.0,273821,...,NaN,alpha-hydroxymetoprolol,Fig3_hmt1,NaN,gram / liter,1.873595,NaN,0.000047,3.0,NaN
211271,211271,Bae2014,thalf,plasma,sample-mean,0.884000,0.510378,NaN,2279.0,273873,...,NaN,alpha-hydroxymetoprolol,NaN,NaN,hour,0.112611,NaN,7.850000,NaN,NaN


==> df has : 211273 rows && 28 columns and the columns name are :**

---



In [ ]:
list(df.columns)

['Unnamed: 0',
 'study_name',
 'measurement_type',
 'tissue',
 'calculation_type',
 'sd',
 'se',
 'min',
 'group_pk',
 'output_pk',
 'time_unit',
 'normed',
 'calculated',
 'value',
 'study_sid',
 'intervention_pk',
 'output_type',
 'method',
 'max',
 'substance',
 'label',
 'individual_pk',
 'unit',
 'cv',
 'median',
 'mean',
 'time',
 'choice']

We are interested in the **concentration** .So, we eliminate any other value of the column 'calculation_type' except 'concentration' :

*   **'calculation_type'= 'concentration'**




In [ ]:
select_concentration = df.loc[df['measurement_type'] == 'concentration']

In [ ]:
select_concentration

,Unnamed: 0,study_name,measurement_type,tissue,calculation_type,sd,se,min,group_pk,output_pk,...,max,substance,label,individual_pk,unit,cv,median,mean,time,choice
18,18,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,247,...,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,NaN,NaN,NaN,7.00000,NaN
21,21,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,238,...,NaN,apapsul-plus-apapglu,fig2_apapsul+apapglu_plasma,2.0,gram / liter,NaN,NaN,NaN,1.00000,NaN
23,23,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,209,...,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,NaN,NaN,NaN,3.00000,NaN
25,25,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,207,...,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,NaN,NaN,NaN,0.50000,NaN
29,29,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,259,...,NaN,apapsul-plus-apapglu,fig3_apapsul+apapglu_plasma,10.0,gram / liter,NaN,NaN,NaN,7.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211263,211263,Bae2014,concentration,plasma,sample-mean,0.000009,0.000005,NaN,2278.0,273791,...,NaN,metoprolol,Fig3_met1,NaN,gram / liter,1.010963,NaN,0.000009,0.34113,NaN
211264,211264,Bae2014,concentration,plasma,sample-mean,0.000075,0.000038,NaN,2278.0,273822,...,NaN,alpha-hydroxymetoprolol,Fig3_hmt1,NaN,gram / liter,1.807531,NaN,0.000042,4.00000,NaN
211268,211268,Bae2014,concentration,plasma,sample-mean,0.000168,0.000097,NaN,2279.0,273813,...,NaN,metoprolol,Fig3_met10,NaN,gram / liter,1.120702,NaN,0.000150,10.00000,NaN
211269,211269,Bae2014,concentration,plasma,sample-mean,0.000031,0.000015,NaN,2278.0,273825,...,NaN,alpha-hydroxymetoprolol,Fig3_hmt1,NaN,gram / liter,1.318690,NaN,0.000023,8.00000,NaN


**Check the column 'value' and delete rows having NaN value**

In [ ]:
print(select_concentration[['value']])

           value
18      0.002008
21      0.030269
23      0.004022
25      0.006407
29      0.041343
...          ...
211263       NaN
211264       NaN
211268       NaN
211269       NaN
211270       NaN

[84342 rows x 1 columns]


In [ ]:
df2=select_concentration.dropna(subset=['value'])

In [ ]:
df2

,Unnamed: 0,study_name,measurement_type,tissue,calculation_type,sd,se,min,group_pk,output_pk,...,max,substance,label,individual_pk,unit,cv,median,mean,time,choice
18,18,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,247,...,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,NaN,NaN,NaN,7.0,NaN
21,21,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,238,...,NaN,apapsul-plus-apapglu,fig2_apapsul+apapglu_plasma,2.0,gram / liter,NaN,NaN,NaN,1.0,NaN
23,23,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,209,...,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,NaN,NaN,NaN,3.0,NaN
25,25,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,207,...,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,NaN,NaN,NaN,0.5,NaN
29,29,Lowenthal1976,concentration,plasma,NaN,NaN,NaN,NaN,NaN,259,...,NaN,apapsul-plus-apapglu,fig3_apapsul+apapglu_plasma,10.0,gram / liter,NaN,NaN,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208770,208770,Lesne1982,concentration,plasma,NaN,NaN,NaN,NaN,NaN,268445,...,NaN,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,NaN,NaN,NaN,0.5,NaN
208785,208785,Lesne1982,concentration,plasma,NaN,NaN,NaN,NaN,NaN,268377,...,NaN,torasemide,fig2_MC_Dtor_po,15289.0,gram / liter,NaN,NaN,NaN,10.0,NaN
208787,208787,Lesne1982,concentration,plasma,NaN,NaN,NaN,NaN,NaN,268427,...,NaN,torasemide,fig2_TV_Dtor_iv,15291.0,gram / liter,NaN,NaN,NaN,2.0,NaN
208802,208802,Lesne1982,concentration,plasma,NaN,NaN,NaN,NaN,NaN,268452,...,NaN,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,NaN,NaN,NaN,6.0,NaN


**Now, we have :**
**   37737 rows × 28 columns**



**delete columns having NaN in all rows**

In [ ]:
df3 = df2.dropna(axis=1, how='all')

In [ ]:
df3

,Unnamed: 0,study_name,measurement_type,tissue,output_pk,time_unit,normed,calculated,value,study_sid,intervention_pk,output_type,method,substance,label,individual_pk,unit,time
18,18,Lowenthal1976,concentration,plasma,247,hr,True,False,0.002008,PKDB00213,2,timecourse,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,7.0
21,21,Lowenthal1976,concentration,plasma,238,hr,True,False,0.030269,PKDB00213,2,timecourse,NaN,apapsul-plus-apapglu,fig2_apapsul+apapglu_plasma,2.0,gram / liter,1.0
23,23,Lowenthal1976,concentration,plasma,209,hr,True,False,0.004022,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,3.0
25,25,Lowenthal1976,concentration,plasma,207,hr,True,False,0.006407,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,0.5
29,29,Lowenthal1976,concentration,plasma,259,hr,True,False,0.041343,PKDB00213,2,timecourse,NaN,apapsul-plus-apapglu,fig3_apapsul+apapglu_plasma,10.0,gram / liter,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208770,208770,Lesne1982,concentration,plasma,268445,hr,True,False,0.003449,PKDB00297,4089,timecourse,hplc,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,0.5
208785,208785,Lesne1982,concentration,plasma,268377,hr,True,False,0.000193,PKDB00297,4090,timecourse,hplc,torasemide,fig2_MC_Dtor_po,15289.0,gram / liter,10.0
208787,208787,Lesne1982,concentration,plasma,268427,hr,True,False,0.001241,PKDB00297,4089,timecourse,hplc,torasemide,fig2_TV_Dtor_iv,15291.0,gram / liter,2.0
208802,208802,Lesne1982,concentration,plasma,268452,hr,True,False,0.000139,PKDB00297,4089,timecourse,hplc,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,6.0


**Check if we have the same unit in all rows :**

In [ ]:
pd.unique(df3[['unit']].values.ravel())

array(['gram / liter', 'mole / liter', 'IU / liter'], dtype=object)

**To convert 'mole / liter' to 'gram / liter' we multiply the concentration by the molar mass = 55,9g.mol-1**



In [ ]:
df3.loc[df['unit'] == 'mole / liter', 'value'] *= 55.9
df3.loc[df['unit'] == 'mole / liter', 'unit'] = 'gram / liter'

**IU is a unit of biological activity, and its conversion to grams depends on the specific substance. As we have only 53 rows, we won't consider these rows**

In [ ]:
len(df3[df3["unit"]=="IU / liter"])

53

**df4 will have the same unit 'gram / liter'**

In [ ]:
len(df3[df3["unit"]=="gram / liter"])

37684

**we will take only the rows having the unit 'gram / liter'**

In [ ]:
df4=df3.loc[df['unit'] == 'gram / liter']

In [ ]:
df4

,Unnamed: 0,study_name,measurement_type,tissue,output_pk,time_unit,normed,calculated,value,study_sid,intervention_pk,output_type,method,substance,label,individual_pk,unit,time
18,18,Lowenthal1976,concentration,plasma,247,hr,True,False,0.002008,PKDB00213,2,timecourse,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,7.0
21,21,Lowenthal1976,concentration,plasma,238,hr,True,False,0.030269,PKDB00213,2,timecourse,NaN,apapsul-plus-apapglu,fig2_apapsul+apapglu_plasma,2.0,gram / liter,1.0
23,23,Lowenthal1976,concentration,plasma,209,hr,True,False,0.004022,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,3.0
25,25,Lowenthal1976,concentration,plasma,207,hr,True,False,0.006407,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,0.5
29,29,Lowenthal1976,concentration,plasma,259,hr,True,False,0.041343,PKDB00213,2,timecourse,NaN,apapsul-plus-apapglu,fig3_apapsul+apapglu_plasma,10.0,gram / liter,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208770,208770,Lesne1982,concentration,plasma,268445,hr,True,False,0.003449,PKDB00297,4089,timecourse,hplc,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,0.5
208785,208785,Lesne1982,concentration,plasma,268377,hr,True,False,0.000193,PKDB00297,4090,timecourse,hplc,torasemide,fig2_MC_Dtor_po,15289.0,gram / liter,10.0
208787,208787,Lesne1982,concentration,plasma,268427,hr,True,False,0.001241,PKDB00297,4089,timecourse,hplc,torasemide,fig2_TV_Dtor_iv,15291.0,gram / liter,2.0
208802,208802,Lesne1982,concentration,plasma,268452,hr,True,False,0.000139,PKDB00297,4089,timecourse,hplc,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,6.0


**We need 100 concentration values. So we will choose a substance : "memantine"**. We have **707** rows

In [ ]:
pd.unique(df4[['substance']].values.ravel())

array(['apap', 'apapsul-plus-apapglu', 'apapglu', 'apapsul',
       'sulfamethizole', 'phenylbutazone', 'salicylic-acid', 'antipyrine',
       'caf', 'metoprolol', 'warfarin', 'omeprazole', 'midazolam',
       'theacrine', 'mor', 'cod', 'px', 'albumin', 'bilirubin', 'tb',
       'tp', 'mexiletine', 'progesterone', 'estradiol', 'dmt', 'dtf',
       'hm3', 'mom3', 'dtf-plus-dtfglu', 'hm3-plus-hm3glu', 'ldlc', 'm3g',
       'c6g', 'm6g', 'nmor', 'digoxin', '1-hydroxymidazolam', 'memantine',
       '5-hydroxyomeprazole', 'omeprazole-sulfone', 'repaglinide',
       'hydroxyrepaglinide', 'rosuvastatin', 'tolbutamide',
       '4-hydroxytolbutamide', 'nordazepam', 'diazepam', 'torasemide',
       'glucose', 'lorazepam', 'icg', 'oxazepam', 'lidocaine',
       'metoh-plus-mam', '1-hydroxymidazolam-glucuronide', 'alfentanil',
       'sulfapyridine', 'oxazepam-glucuronide', 'pra', 'cholesterol'],
      dtype=object)

In [ ]:
#how many rows for substance paracetamol apap
len(df4[df4["substance"]=="apap"])

3220

In [ ]:
select_substance = df4.loc[df4['substance'] == 'apap']
select_substance

,Unnamed: 0,study_name,measurement_type,tissue,output_pk,time_unit,normed,calculated,value,study_sid,intervention_pk,output_type,method,substance,label,individual_pk,unit,time
18,18,Lowenthal1976,concentration,plasma,247,hr,True,False,0.002008,PKDB00213,2,timecourse,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,7.000000
23,23,Lowenthal1976,concentration,plasma,209,hr,True,False,0.004022,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,3.000000
25,25,Lowenthal1976,concentration,plasma,207,hr,True,False,0.006407,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,0.500000
35,35,Lowenthal1976,concentration,plasma,226,hr,True,False,0.012983,PKDB00213,2,timecourse,NaN,apap,fig2_paracetamol_plasma,2.0,gram / liter,1.000000
40,40,Lowenthal1976,concentration,plasma,248,hr,True,False,0.001229,PKDB00213,2,timecourse,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,9.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200255,200255,Villesen2007b,concentration,plasma,236373,hr,True,False,0.000629,17541571,3593,timecourse,NaN,apap,S4_paracetamol,14337.0,gram / liter,11.901504
200257,200257,Villesen2007b,concentration,plasma,236584,hr,True,False,0.003440,17541571,3593,timecourse,NaN,apap,S7_paracetamol,14340.0,gram / liter,3.823036
200324,200324,Villesen2007b,concentration,plasma,236585,hr,True,False,0.002212,17541571,3594,timecourse,NaN,apap,S7_paracetamol,14340.0,gram / liter,4.915172
200327,200327,Villesen2007b,concentration,plasma,236585,hr,True,False,0.002212,17541571,3588,timecourse,NaN,apap,S7_paracetamol,14340.0,gram / liter,4.915172


In [ ]:
pd.unique(select_substance[['time_unit']].values.ravel())

array(['hr', 'min'], dtype=object)

In [ ]:
select_substance.loc[df['time_unit'] == 'min', 'time'] /= 60

In [ ]:
select_substance.loc[df['time_unit'] == 'min', 'time_unit'] = 'hr'

In [ ]:
pd.unique(select_substance[['time_unit']].values.ravel())

array(['hr'], dtype=object)

In [ ]:
#pd.unique(select_substance[['label']].values.ravel())

In [ ]:
#len(select_substance[select_substance["label"]=="Fig2_memantine_2"])==> si non on prends ce label et au lieu de 100 row ywaliw 80 rows in the  code

In [ ]:
pd.unique(select_substance[['value']].values.ravel())################################################################

array([0.00200771, 0.00402179, 0.00640663, ..., 0.00019201, 0.00344042,
       0.00114163])

In [ ]:
final_2_columns_with_duplicate_time = select_substance[['value', 'time']]# select the 2 columns

In [ ]:
final_2_columns= final_2_columns_with_duplicate_time.drop_duplicates(subset=['time'])#drop dupicate time values
len(final_2_columns)

353

In [ ]:
final_2_columns

,value,time
18,0.002008,7.000000
23,0.004022,3.000000
25,0.006407,0.500000
35,0.012983,1.000000
40,0.001229,9.000000
...,...,...
189348,0.002212,4.915172
189508,0.001737,7.945206
189516,0.000192,19.962620
189588,0.003440,3.823036


In [ ]:
value_time_sorted = final_2_columns.sort_values(by='time')
value_time_sorted

,value,time
1193,0.025429,0.000000
4857,0.007604,0.047599
2602,0.002584,0.083000
1243,0.026278,0.083333
4778,0.004977,0.087699
...,...,...
187856,0.000021,48.153214
169778,0.000049,49.589043
186948,0.000015,60.054718
167730,0.000034,60.410957


In [ ]:
final_100_rows = value_time_sorted.head(100)#select 100 first rows

In [ ]:
final_100_rows

,value,time
1193,0.025429,0.000000
4857,0.007604,0.047599
2602,0.002584,0.083000
1243,0.026278,0.083333
4778,0.004977,0.087699
...,...,...
105558,0.000655,1.250000
10677,0.010962,1.256939
10875,0.015811,1.259718
10686,0.010979,1.261693


**Convert data to csv and download it**

In [ ]:
final_100_rows.to_csv('ConcentrationTime.csv', index=False)

In [ ]:
value_concentration = final_100_rows[['value']] #only "value" column
value_concentration

,value
1193,0.025429
4857,0.007604
2602,0.002584
1243,0.026278
4778,0.004977
...,...
105558,0.000655
10677,0.010962
10875,0.015811
10686,0.010979


In [ ]:
value_concentration.to_csv('value_concentration.csv', index=False)

In [ ]:
time = final_100_rows[['time']] #only "time" column
time

,time
1193,0.000000
4857,0.047599
2602,0.083000
1243,0.083333
4778,0.087699
...,...
105558,1.250000
10677,1.256939
10875,1.259718
10686,1.261693


In [ ]:
time.to_csv('time.csv', index=False)

**Check if we have the same 'time_unit' in all rows :**

In [ ]:
pd.unique(select_substance[['time_unit']].values.ravel())

array(['hr'], dtype=object)

# Trying to have the same "time_unit" for earlier dataframe "df4"

In [ ]:
#pd.unique(df4[['time_unit']].values.ravel())

In [ ]:
#df4=df3.dropna(subset=['time_unit'])

In [ ]:
#pd.unique(df4[['time_unit']].values.ravel())

array(['hr', 'min', 'weeks', 'days', 'day', 'week'], dtype=object)

In [ ]:
#df4.loc[df['time_unit'] == 'min', 'time'] /= 60
#df4.loc[df['time_unit'] == 'weeks', 'time'] *= 168
#df4.loc[df['time_unit'] == 'days', 'time'] *= 24
#df4.loc[df['time_unit'] == 'day', 'time'] *= 24
#df4.loc[df['time_unit'] == 'week', 'time'] *= 168

In [ ]:
#df4.loc[df['time_unit'] == 'min', 'time_unit'] = 'hr'
#df4.loc[df['time_unit'] == 'weeks', 'time_unit'] = 'hr'
#df4.loc[df['time_unit'] == 'days', 'time_unit'] = 'hr'
#df4.loc[df['time_unit'] == 'day', 'time_unit'] = 'hr'
#df4.loc[df['time_unit'] == 'week', 'time_unit'] = 'hr'

In [ ]:
#pd.unique(df4[['time_unit']].values.ravel())

array(['hr'], dtype=object)

In [ ]:
#df4

,Unnamed: 0,study_name,measurement_type,tissue,output_pk,time_unit,normed,calculated,value,study_sid,intervention_pk,output_type,method,substance,label,individual_pk,unit,time
18,18,Lowenthal1976,concentration,plasma,247,hr,True,False,0.002008,PKDB00213,2,timecourse,NaN,apap,fig3_paracetamol_plasma,10.0,gram / liter,7.0
21,21,Lowenthal1976,concentration,plasma,238,hr,True,False,0.030269,PKDB00213,2,timecourse,NaN,apapsul-plus-apapglu,fig2_apapsul+apapglu_plasma,2.0,gram / liter,1.0
23,23,Lowenthal1976,concentration,plasma,209,hr,True,False,0.004022,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,3.0
25,25,Lowenthal1976,concentration,plasma,207,hr,True,False,0.006407,PKDB00213,2,timecourse,NaN,apap,fig1_paracetamol_plasma,12.0,gram / liter,0.5
29,29,Lowenthal1976,concentration,plasma,259,hr,True,False,0.041343,PKDB00213,2,timecourse,NaN,apapsul-plus-apapglu,fig3_apapsul+apapglu_plasma,10.0,gram / liter,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208770,208770,Lesne1982,concentration,plasma,268445,hr,True,False,0.003449,PKDB00297,4089,timecourse,hplc,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,0.5
208785,208785,Lesne1982,concentration,plasma,268377,hr,True,False,0.000193,PKDB00297,4090,timecourse,hplc,torasemide,fig2_MC_Dtor_po,15289.0,gram / liter,10.0
208787,208787,Lesne1982,concentration,plasma,268427,hr,True,False,0.001241,PKDB00297,4089,timecourse,hplc,torasemide,fig2_TV_Dtor_iv,15291.0,gram / liter,2.0
208802,208802,Lesne1982,concentration,plasma,268452,hr,True,False,0.000139,PKDB00297,4089,timecourse,hplc,torasemide,fig2_LM_Dtor_iv,15292.0,gram / liter,6.0
